<a href="https://colab.research.google.com/github/urvashiramdasani/Document-Summarization/blob/main/notebooks/20_newsgroups_using_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook presents the code for 20 Newsgroups dataset document summarization.

## Setting up GitHub

In [ ]:
!git clone https://github.com/urvashiramdasani/Document-Summarization.git

Cloning into 'Document-Summarization'...
remote: Enumerating objects: 18952, done.
remote: Counting objects: 100% (18952/18952), done.
remote: Compressing objects: 100% (18917/18917), done.
remote: Total 18952 (delta 60), reused 18893 (delta 33), pack-reused 0
Receiving objects: 100% (18952/18952), 19.53 MiB | 11.83 MiB/s, done.
Resolving deltas: 100% (60/60), done.


## Data Preprocessing

In [ ]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    # filedata = filedata[26:50]
    sentences = []

    for sentence in filedata:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").strip(" \n"))
    
    return sentences

In [ ]:
# Reading a sample article

sentences = read_article("/content/001.txt")
sentences = list(filter(None, sentences))
print(sentences)

['Ad sales boost Time Warner profit', 'Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.', 'The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.', "Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sig

In [ ]:
len(sentences)

6

## POS Tagging

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
txt = ""
for sentence in sentences:
  txt += "".join(sentence)
  txt += " "
print(txt)

MARINER 1, the first U.S. attempt to send a spacecraft to Venus, failed minutes after launch in 1962. The guidance instructions from the ground stopped reaching the rocket due to a problem with its antenna, so the onboard computer took control. However, there turned out to be a bug in the guidance software, and the rocket promptly went off course, so the Range Safety Officer destroyed it. Although the bug is sometimes claimed to have been an incorrect FORTRAN DO statement, it was actually a transcription error in which the bar (indicating smoothing) was omitted from the expression "R-dot-bar sub n" (nth smoothed value of derivative of radius). This error led the software to treat normal minor variations of velocity as if they were serious, leading to incorrect compensation. MARINER 2 became the first successful probe to flyby Venus in December of 1962, and it returned information which confirmed that Venus is a very hot (800 degrees Fahrenheit, now revised to 900 degrees F.) world with

In [ ]:
tokenized = sent_tokenize(txt)
tagged = []
for i in tokenized:
    wordsList = nltk.word_tokenize(i)
    wordsList = [w for w in wordsList if not w in stop_words] 
    tagged.append(nltk.pos_tag(wordsList))

In [ ]:
print(tagged)

[[('This', 'DT'), ('section', 'NN'), ('lightly', 'RB'), ('adapted', 'VBD'), ('original', 'JJ'), ('posting', 'NN'), ('Larry', 'NNP'), ('Klaes', 'NNP'), ('(', '('), ('klaes', 'VB'), ('@', 'NNP'), ('verga.enet.dec.com', 'NN'), (')', ')'), (',', ','), ('mostly', 'RB'), ('minor', 'JJ'), ('formatting', 'NN'), ('changes', 'NNS'), ('.', '.')], [('Matthew', 'NNP'), ('Wiener', 'NNP'), ('(', '('), ('weemba', 'JJ'), ('@', 'NNP'), ('libra.wistar.upenn.edu', 'NN'), (')', ')'), ('contributed', 'VBD'), ('section', 'NN'), ('Voyager', 'NNP'), (',', ','), ('section', 'NN'), ('Sakigake', 'NNP'), ('obtained', 'VBD'), ('ISAS', 'NNP'), ('material', 'NN'), ('posted', 'VBD'), ('Yoshiro', 'NNP'), ('Yamada', 'NNP'), ('(', '('), ('yamada', 'PRP'), ('@', 'NNP'), ('yscvax.ysc.go.jp', 'NN'), (')', ')'), ('.', '.')], [('US', 'NNP'), ('PLANETARY', 'NNP'), ('MISSIONS', 'NNP'), ('MARINER', 'NNP'), ('(', '('), ('VENUS', 'NNP'), (',', ','), ('MARS', 'NNP'), (',', ','), ('&', 'CC'), ('MERCURY', 'NNP'), ('FLYBYS', 'NNP'), (

In [ ]:
annotations = set()

for i in range(len(tagged)):
  for j in range(len(tagged[i])):
    annotations.add(tagged[i][j][1])

print(annotations)

{'POS', 'CD', 'VBZ', 'DT', 'FW', '.', 'PRP', 'JJR', '``', "''", 'VBP', 'MD', 'VBN', '$', ',', 'WRB', 'JJS', 'PRP$', 'NN', '#', 'NNP', 'NNPS', 'RB', 'VBG', 'JJ', 'IN', '(', 'RBR', 'NNS', 'RP', 'VBD', 'CC', ')', 'VB', ':', 'WP'}


In [ ]:
tagged[0][0][1]

'NNP'

In [ ]:
corpus = ""
for i in range(len(tagged)):
  for j in range(len(tagged[i])):
    if tagged[i][j][1] != 'JJ' and tagged[i][j][1] != 'DT':
      corpus += tagged[i][j][0] + " "

print(corpus)

section lightly adapted posting Larry Klaes ( klaes @ verga.enet.dec.com ) , mostly formatting changes . Matthew Wiener ( @ libra.wistar.upenn.edu ) contributed section Voyager , section Sakigake obtained ISAS material posted Yoshiro Yamada ( yamada @ yscvax.ysc.go.jp ) . US PLANETARY MISSIONS MARINER ( VENUS , MARS , & MERCURY FLYBYS AND ORBITERS ) MARINER 1 , U.S. attempt send spacecraft Venus , failed minutes 1962 . guidance instructions ground stopped reaching rocket problem antenna , onboard computer took control . However , turned guidance software , rocket promptly went course , Range Safety Officer destroyed . Although bug sometimes claimed FORTRAN DO statement , actually transcription error bar ( indicating smoothing ) omitted expression `` sub n '' ( nth smoothed value radius ) . error led software treat variations velocity , compensation . MARINER 2 became first probe flyby Venus December 1962 , returned information confirmed Venus ( 800 degrees Fahrenheit , revised 900 degr

## Hidden Markov Models



In [ ]:
CONST = 2
n = 4
threshold = 1.0e-120

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils, to_categorical
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, LSTM
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
# Preprocess data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab = tokenizer.word_index
seqs = tokenizer.texts_to_sequences(sentences)

In [ ]:
def prepare_sentence(seq, maxlen):
    # Pads seq and slides windows
    x = []
    y = []
    for i, w in enumerate(seq):
        x_padded = pad_sequences([seq[:i]],
                                 maxlen=maxlen - 1,
                                 padding='pre')[0]  # Pads before each sequence
        x.append(x_padded)
        y.append(w)
    return x, y

# Pad sequences and slide windows
maxlen = max([len(seq) for seq in seqs])
x = []
y = []
for seq in seqs:
    x_windows, y_windows = prepare_sentence(seq, maxlen)
    x += x_windows
    y += y_windows
x = np.array(x)
y = np.array(y) - 1  # The word <PAD> does not constitute a class
y = np.eye(len(vocab))[y]  # One hot encoding

In [ ]:
# Define model
model = Sequential()
model.add(Embedding(input_dim=len(vocab) + 1,  # vocabulary size. Adding an
                                               # extra element for <PAD> word
                    output_dim=5,  # size of embeddings
                    input_length=maxlen - 1))  # length of the padded sequences
model.add(LSTM(10))
model.add(Dense(len(vocab), activation='softmax'))
model.compile('rmsprop', 'categorical_crossentropy', metrics = ['accuracy'])

# Train network
model.fit(x, y, epochs=100)

Epoch 1/100
8/8 [==============================] - 2s 6ms/step - loss: 5.0752 - accuracy: 0.0129 
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0699 - accuracy: 0.0854
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0667 - accuracy: 0.0582
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0615 - accuracy: 0.0754
Epoch 5/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0577 - accuracy: 0.0488
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0482 - accuracy: 0.0670
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0411 - accuracy: 0.0587
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0270 - accuracy: 0.0638
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 5.0025 - accuracy: 0.0498
Epoch 10/100
8/8 [==============================] - 0s 6ms/step - loss: 4.9614 - accuracy: 0.0732
Epoch 11/100
8/8 [==========

In [ ]:
# Compute probability of occurence of a sentence
def probability_of_sentence(sentence):
  tok = tokenizer.texts_to_sequences([sentence])[0]
  x_test, y_test = prepare_sentence(tok, maxlen)
  x_test = np.array(x_test)
  y_test = np.array(y_test) - 1  # The word <PAD> does not constitute a class
  p_pred = model.predict(x_test)  # array of conditional probabilities
  vocab_inv = {v: k for k, v in vocab.items()}

  # Compute product
  # Efficient version: np.exp(np.sum(np.log(np.diag(p_pred[:, y_test]))))
  log_p_sentence = 0
  for i, prob in enumerate(p_pred):
      word = vocab_inv[y_test[i]+1]  # Index 0 from vocab is reserved to <PAD>
      history = ' '.join([vocab_inv[w] for w in x_test[i, :] if w != 0])
      prob_word = prob[y_test[i]]
      log_p_sentence += np.log(prob_word)
      # print('P(w={}|h={})={}'.format(word, history, prob_word))
  return np.exp(log_p_sentence)

In [ ]:
import pandas as pd

df = pd.DataFrame(columns = ['Sentence', 'Probability'])

In [ ]:
# txt = txt.split(" .")
word_list = ['', '', '', '', '', '', '', '']

for i in range(len(sentences)):
  words = sentences[i].split(" .")
  num_words = len(words)
  for j in range(num_words):
    word_list[3] = words[j]
    if j + n <= num_words:
      word_list[5] = words[j + n]
    if j - n >= 0:
      word_list[2] = words[j - n]
    if j + 1 < num_words:
      word_list[4] = words[j + 1]
    if i - CONST >= 0:
      word_list[0] = sentences[i - CONST].split(" .")[j]
    if i - CONST + 1>= 0:
      word_list[1] = sentences[i - CONST + 1].split(" .")[j]
    if i + CONST < len(sentences):
      word_list[7] = sentences[i + CONST].split(" .")[j]
    if i + CONST - 1 < len(sentences):
      word_list[6] = sentences[i + CONST - 1].split(" .")[j]
    final_str = " ".join(word_list)
    df = df.append({'Sentence':final_str, 'Probability':probability_of_sentence(final_str)}, ignore_index = True)

In [ ]:
df

,Sentence,Probability
0,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is no...",9.782486e-53
1,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now...",3.001788e-117
2,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now on...",5.195843e-184
3,"Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Googl...",6.817101e-283
4,"The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11....",0.000000e+00
5,"Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were l...",0.000000e+00


In [ ]:
df.describe()

,Probability
count,6.000000e+00
mean,1.630414e-53
std,3.993683e-53
min,0.000000e+00
25%,1.704275e-283
50%,2.597922e-184
75%,2.251341e-117
max,9.782486e-53


In [ ]:
# Filter out all the sentences whose probability > 1.0e-140
df = df[df.Probability > 1.0e-117]

In [ ]:
df

,Sentence,Probability
0,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is no...",9.782486e-53
1,"Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now...",3.001788e-117


In [ ]:
hmm_summary = ""

for sentence in df['Sentence']:
  hmm_summary += sentence

In [ ]:
!pip install rouge-score

In [ ]:
human_summary = "TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations."

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3','rougeL'], use_stemmer=True)
scores = scorer.score(human_summary, hmm_summary)

In [ ]:
scores

{'rouge1': Score(precision=0.42758620689655175, recall=0.8104575163398693, fmeasure=0.5598194130925509),
 'rouge2': Score(precision=0.33217993079584773, recall=0.631578947368421, fmeasure=0.43537414965986393),
 'rouge3': Score(precision=0.2986111111111111, recall=0.5695364238410596, fmeasure=0.39179954441913434),
 'rougeL': Score(precision=0.2896551724137931, recall=0.5490196078431373, fmeasure=0.3792325056433409)}

In [ ]:
hmm_summary

"   Ad sales boost Time Warner profit   Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Ad sales boost Time Warner profit  Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.   The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL. Time Warn

## BERT

In [ ]:
!pip install bert-extractive-summarizer==0.4.2

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 3.3MB 44.5MB/s 
     |████████████████████████████████| 901kB 36.4MB/s 
  Created wheel for bert-extractive-summarizer: filename=bert_extractive_summarizer-0.4.2-cp37-none-any.whl size=13711 sha256=0f44f75478f766a2498908bd3cfa4988f47b4e44ea867b297c1ab9e505a27cc7
  Stored in directory: /root/.cache/pip/wheels/13/bc/30/654eb9e657177a56cba927c5a20b6cd01fb229b1ed2bf9b371
Successfully built bert-extractive-summarizer


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.4MB/s 


In [ ]:
import torch
from summarizer import Summarizer

In [ ]:
model = Summarizer('distilbert-base-uncased')

In [ ]:
resp = model(txt)
print(resp)

MARINER 1, the first U.S. attempt to send a spacecraft to Venus, failed minutes after launch in 1962. MARINER 2 became the first successful probe to flyby Venus in December of 1962, and it returned information which confirmed that Venus is a very hot (800 degrees Fahrenheit, now revised to 900 degrees F.) world with a cloud-covered atmosphere composed primarily of carbon dioxide (sulfuric acid was later confirmed in 1978).


In [ ]:
print(len(resp), len(txt))

426 1445


In [ ]:
print(scorer.score(human_summary, resp))

{'rouge1': Score(precision=0.75, recall=0.3673469387755102, fmeasure=0.49315068493150693), 'rouge2': Score(precision=0.38028169014084506, recall=0.18493150684931506, fmeasure=0.24884792626728108), 'rouge3': Score(precision=0.21428571428571427, recall=0.10344827586206896, fmeasure=0.1395348837209302), 'rougeL': Score(precision=0.625, recall=0.30612244897959184, fmeasure=0.4109589041095891)}


## Abstractive Summarization

In [ ]:
# tqdm version 4.36.1 is required


!pip install tqdm==4.36.1

     |████████████████████████████████| 61kB 2.3MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.36.1 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
# Importing Libaries


import numpy as np  
import pandas as pd 
import re           
import glob
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import keras
import warnings
import nltk
nltk.download('stopwords')
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Progress bar


from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Preprocessing "body" text


from spacy.lang.en.stop_words import STOP_WORDS
 
def clean_body(text):
    newText = text.lower()
    newText = re.sub('[^\w\s\d\.]','',newText)
    newText = ' '.join(newText.split())
    tokens = [w for w in newText.split() if not w in STOP_WORDS]
    long_words=[]
    for i in tokens:
        if len(i)>=3:
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [ ]:
clean_txt = clean_body(txt)

In [ ]:
clean_txt

'mariner u.s. attempt send spacecraft venus failed minutes launch 1962. guidance instructions ground stopped reaching rocket problem antenna onboard computer took control. turned bug guidance software rocket promptly went course range safety officer destroyed it. bug claimed incorrect fortran statement actually transcription error bar indicating smoothing omitted expression rdotbar sub nth smoothed value derivative radius. error led software treat normal minor variations velocity leading incorrect compensation. mariner successful probe flyby venus december 1962 returned information confirmed venus hot 800 degrees fahrenheit revised 900 degrees world cloudcovered atmosphere composed primarily carbon dioxide sulfuric acid later confirmed 1978. mariner launched november 1964 lost protective shroud failed eject craft placed interplanetary space. unable collect suns energy power solar panels probe soon died batteries ran solar orbit. intended mars flyby mariner'

In [ ]:
# Preprocessing "highlight" text


def clean_highlight(text):
  newText = text.lower()
  newText = re.sub('[^\w\s\d\.]','',newText)
  newText = ' '.join(newText.split())
  newText = '_START_ '+ newText + ' _END_'
  return newText

In [ ]:
highlight = "Mariner1 was the first attempt to send a spacecraft to venus, which failed because of fault in antenna guidance instructions couldn't reach the rocket. Hence, the control was taken over onboard computer that had bug in software due to which even the slightest change in the velocity were considered significant leading to incorrect compensation. In December 1962, MARINER 2 became the successful probe to  venus and it return the information confirming venus is very hot, nearly 800 to 900 Fahrenheit with cloud covered atmosphere comprising of carbon dioxide. Sulfuric acid was confirmed later in 1978 On November 5 1964, MARINER3 was launched, but after being placed in the space, failed to eject from the protective shroud. Due to this, it couldn't get solar power for its panels and the probe died after running out of battery. It was intended for Mars fly y with MARINER4"

In [ ]:
clean_highlight_txt = clean_highlight(highlight)

In [ ]:
clean_highlight_txt

'_START_ mariner1 was the first attempt to send a spacecraft to venus which failed because of fault in antenna guidance instructions couldnt reach the rocket. hence the control was taken over onboard computer that had bug in software due to which even the slightest change in the velocity were considered significant leading to incorrect compensation. in december 1962 mariner 2 became the successful probe to venus and it return the information confirming venus is very hot nearly 800 to 900 fahrenheit with cloud covered atmosphere comprising of carbon dioxide. sulfuric acid was confirmed later in 1978 on november 5 1964 mariner3 was launched but after being placed in the space failed to eject from the protective shroud. due to this it couldnt get solar power for its panels and the probe died after running out of battery. it was intended for mars fly y with mariner4 _END_'

In [ ]:
# Splitting data into training and test sets
# Test set is 20% of total data


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(clean_txt, clean_highlight_txt, test_size=0.2, random_state=0, shuffle=True)

ValueError: ignored

In [ ]:
max_len_body = 1000
max_len_highlight = 50

In [ ]:
# Tokenizing "body"
x_tok = Tokenizer()
x_tok.fit_on_texts(list(clean_txt))

# Converting text to number sequences
x_train = x_tok.texts_to_sequences(clean_txt) 
x_test = x_tok.texts_to_sequences(clean_highlight_txt)

# Padding zero upto maximum length
x_train = pad_sequences(x_train,  maxlen=max_len_body, padding='post') 
x_test = pad_sequences(x_test, maxlen=max_len_body, padding='post')

# Total number of words
x_vocab_size = len(x_tok.word_index) +1

In [ ]:
x_train.shape

(969, 1000)

In [ ]:
x_test.shape

(879, 1000)

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
from keras import backend as K 


K.clear_session() 
latent_dim = 50 


# Encoder 
encoder_inputs = Input(shape=(max_len_body,)) 
enc_emb = Embedding(x_vocab_size, latent_dim,trainable=True)(encoder_inputs) 

# 1st LSTM Layer
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

# 2nd LSTM Layer
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

# 3rd LSTM Layer
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Decoder 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(x_vocab_size, latent_dim,trainable=True) 
dec_emb = dec_emb_layer(decoder_inputs) 

# LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

# Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# Dense layer
decoder_dense = TimeDistributed(Dense(x_vocab_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Model Definition
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 50)     1650        input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 1000, 50), ( 20200       embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
history = model.fit(x_train[:879,:], x_test, epochs = 30, batch_size = 128)

Epoch 1/30


ValueError: ignored

## PyTorch

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 3.7MB/s 
     |████████████████████████████████| 3.3MB 12.2MB/s 
     |████████████████████████████████| 901kB 26.7MB/s 


In [ ]:
from transformers import pipeline
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization")

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
summary_text = summarizer(txt, max_length=100, min_length=5, do_sample=False)[0]['summary_text']
print(summary_text)

the first attempt to send a spacecraft to Venus failed minutes after launch in 1962 . the rocket promptly went off course, so the Range Safety Officer destroyed it . a transcription error led the software to treat normal minor variations of velocity as if they were serious .


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3','rougeL'], use_stemmer=True)
scores = scorer.score(human_summary, summary_text)

In [ ]:
scores

{'rouge1': Score(precision=0.5217391304347826, recall=0.16326530612244897, fmeasure=0.24870466321243523),
 'rouge2': Score(precision=0.2, recall=0.06164383561643835, fmeasure=0.09424083769633508),
 'rouge3': Score(precision=0.1590909090909091, recall=0.04827586206896552, fmeasure=0.07407407407407407),
 'rougeL': Score(precision=0.391304347826087, recall=0.12244897959183673, fmeasure=0.18652849740932645)}